# Regression Model

In [72]:
import pandas as pd
import numpy as np
import missingno as msno

import warnings
warnings.filterwarnings('ignore')

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


## Import data

In [73]:
df_click_purchase = pd.read_csv('data/clickstreams_with_purchase.csv')
df_click_purchase.head()

,customer_id,date,time,page_viewed,product_id,ordered
0,1436364061643577713,2018-07-16,14:37:44,HOMEPAGE,0,0.0
1,1436364061643577713,2018-07-16,14:40:44,PRODUCTSIMPLESKU,205401096165894320,0.0
2,1436364061643577713,2018-07-16,14:42:39,PRODUCTSIMPLESKU,9220344733964903803,0.0
3,1436364061643577713,2018-07-16,14:42:53,PRODUCTSIMPLESKU,9220344733964903803,0.0
4,1436364061643577713,2018-07-16,14:44:46,KEYWORDSEARCH,0,0.0


In [74]:
df_orders = pd.read_csv('data/orders.csv')
print(df_orders.shape)
df_orders.head()

(137537, 9)


,order_product_id,order_id,wayfair_product_id,wayfair_website_id,order_date,order_time,customer_id,customer_zip,quantity_ordered
0,8.344710e+18,1.073040e+17,8.089580e+18,Wayfair,7/19/18,9:48:02,3.243950e+18,29464,1
1,7.897050e+18,9.115090e+17,4.407090e+18,Wayfair,7/21/18,15:04:23,7.606760e+18,10928,1
2,3.742700e+17,9.115090e+17,1.327620e+18,Wayfair,7/21/18,15:04:23,7.606760e+18,10928,1
3,7.072690e+18,5.760940e+18,7.293640e+18,Wayfair,7/22/18,21:56:42,3.106940e+18,6712,1
4,7.307240e+18,2.708890e+18,7.264500e+18,Wayfair,7/21/18,12:55:20,1.390570e+18,2468,1


In [75]:
df_orders1 = df_orders.rename(columns={'wayfair_product_id':'product_id'})
df_orders1.head()

,order_product_id,order_id,product_id,wayfair_website_id,order_date,order_time,customer_id,customer_zip,quantity_ordered
0,8.344710e+18,1.073040e+17,8.089580e+18,Wayfair,7/19/18,9:48:02,3.243950e+18,29464,1
1,7.897050e+18,9.115090e+17,4.407090e+18,Wayfair,7/21/18,15:04:23,7.606760e+18,10928,1
2,3.742700e+17,9.115090e+17,1.327620e+18,Wayfair,7/21/18,15:04:23,7.606760e+18,10928,1
3,7.072690e+18,5.760940e+18,7.293640e+18,Wayfair,7/22/18,21:56:42,3.106940e+18,6712,1
4,7.307240e+18,2.708890e+18,7.264500e+18,Wayfair,7/21/18,12:55:20,1.390570e+18,2468,1


In [76]:
df_products = pd.read_csv('data/products.csv')
print(df_products.shape)
df_products.head()

(498673, 13)


,website_name,product_id,product_name,category_name,description,manufacturer_name,weight,dimension_1,dimension_2,dimension_3,num_reviews,onsite_price,on_promotion
0,Joss US,2.790000e+18,Westgate Upholstered Panel Bed,Beds,The 1882 upholstered bed is sure to appeal to ...,Pulaski Furniture,116.0,59.25,87.50,68.75,7.0,409.99,0
1,Joss US,8.190000e+18,3 Piece Leah Bowl Set,"Boxes, Bins, Baskets, & Buckets",Stack green apples on the breakfast bar or dis...,Three Hands Co.,30.8,23.00,28.35,14.17,0.0,89.99,1
2,Joss US,7.240000e+18,Butter Cups/Ranunculus Sunshine Bulb,Planting & Seeding,Start your own backyard garden or add spring s...,Van Zyverden Inc,1.0,16.00,3.00,4.00,0.0,16.99,0
3,Joss US,4.110000e+18,Freesias Double Blooming Bulb,Planting & Seeding,Start your own backyard garden or add spring s...,Van Zyverden Inc,1.0,24.00,2.00,2.00,2.0,9.99,0
4,Joss US,7.670000e+18,Grand Freesia/Ranunculus Blend Bulb,Planting & Seeding,Start your own backyard garden or add spring s...,Van Zyverden Inc,1.0,36.00,3.00,2.00,2.0,9.99,0


In [77]:
df_taxes = pd.read_csv('data/taxes.csv')
df_taxes.head()

,state_fips,state,zipcode,agi_stub,num_returns,num_single,num_joint,num_hoh,num_dependents,num_elderly,...,N19300,A19300,N19500,A19500,N04800,A04800,N07220,A07220,N11901,A11901
0,1,AL,0,1,815440,477700,105350,221200,491310,150920,...,25020,5950,46930,74541,17910,82471,27490,15002,0,0
1,1,AL,0,2,495830,211930,142340,128890,360480,109680,...,64360,17611,102740,243301,44430,292180,43580,45248,0,0
2,1,AL,0,3,263390,83420,137870,36340,182880,75570,...,68320,19479,99920,337719,37110,278272,22780,29166,0,0
3,1,AL,0,4,167190,29420,124060,10610,130160,51540,...,58540,18167,79980,362811,24780,202113,17250,22630,0,0
4,1,AL,0,5,217440,20240,188080,4880,195990,68080,...,117980,44270,151920,1065202,34030,298272,20970,29469,390,155


## Extract features from df_product

In [78]:
columns = ['product_id','weight','num_reviews','onsite_price','on_promotion','description','category_name','manufacturer_name']
df_products1 = df_products.loc[:,columns]
df_products1.head()

,product_id,weight,num_reviews,onsite_price,on_promotion,description,category_name,manufacturer_name
0,2.790000e+18,116.0,7.0,409.99,0,The 1882 upholstered bed is sure to appeal to ...,Beds,Pulaski Furniture
1,8.190000e+18,30.8,0.0,89.99,1,Stack green apples on the breakfast bar or dis...,"Boxes, Bins, Baskets, & Buckets",Three Hands Co.
2,7.240000e+18,1.0,0.0,16.99,0,Start your own backyard garden or add spring s...,Planting & Seeding,Van Zyverden Inc
3,4.110000e+18,1.0,2.0,9.99,0,Start your own backyard garden or add spring s...,Planting & Seeding,Van Zyverden Inc
4,7.670000e+18,1.0,2.0,9.99,0,Start your own backyard garden or add spring s...,Planting & Seeding,Van Zyverden Inc


In [79]:
df = pd.merge(df_orders1, df_products1, on='product_id')
df = df.drop(columns=['order_product_id','order_id','wayfair_website_id'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80123 entries, 0 to 80122
Data columns (total 13 columns):
product_id           80123 non-null float64
order_date           80123 non-null object
order_time           80123 non-null object
customer_id          80123 non-null float64
customer_zip         80123 non-null object
quantity_ordered     80123 non-null int64
weight               80123 non-null float64
num_reviews          80123 non-null float64
onsite_price         80123 non-null float64
on_promotion         80123 non-null int64
description          80123 non-null object
category_name        80123 non-null object
manufacturer_name    80123 non-null object
dtypes: float64(5), int64(2), object(6)
memory usage: 8.6+ MB


In [80]:
df['order_value'] = df.quantity_ordered*df.onsite_price
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80123 entries, 0 to 80122
Data columns (total 14 columns):
product_id           80123 non-null float64
order_date           80123 non-null object
order_time           80123 non-null object
customer_id          80123 non-null float64
customer_zip         80123 non-null object
quantity_ordered     80123 non-null int64
weight               80123 non-null float64
num_reviews          80123 non-null float64
onsite_price         80123 non-null float64
on_promotion         80123 non-null int64
description          80123 non-null object
category_name        80123 non-null object
manufacturer_name    80123 non-null object
order_value          80123 non-null float64
dtypes: float64(6), int64(2), object(6)
memory usage: 9.2+ MB


## Features int df_taxes

In [81]:
df.customer_zip=df.customer_zip.astype(int)
df = df.rename(columns={'customer_zip':'zipcode'})



In [82]:
df_taxes1 = df_taxes.loc[:,['state','zipcode','num_returns']]
df_taxes1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179796 entries, 0 to 179795
Data columns (total 3 columns):
state          179796 non-null object
zipcode        179796 non-null int64
num_returns    179796 non-null int64
dtypes: int64(2), object(1)
memory usage: 4.1+ MB


In [83]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80123 entries, 0 to 80122
Data columns (total 14 columns):
product_id           80123 non-null float64
order_date           80123 non-null object
order_time           80123 non-null object
customer_id          80123 non-null float64
zipcode              80123 non-null int64
quantity_ordered     80123 non-null int64
weight               80123 non-null float64
num_reviews          80123 non-null float64
onsite_price         80123 non-null float64
on_promotion         80123 non-null int64
description          80123 non-null object
category_name        80123 non-null object
manufacturer_name    80123 non-null object
order_value          80123 non-null float64
dtypes: float64(6), int64(3), object(5)
memory usage: 9.2+ MB


In [84]:
df_taxes1=df_taxes1.groupby('zipcode').mean()
df = pd.merge(df,df_taxes1,on='zipcode')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79080 entries, 0 to 79079
Data columns (total 15 columns):
product_id           79080 non-null float64
order_date           79080 non-null object
order_time           79080 non-null object
customer_id          79080 non-null float64
zipcode              79080 non-null int64
quantity_ordered     79080 non-null int64
weight               79080 non-null float64
num_reviews          79080 non-null float64
onsite_price         79080 non-null float64
on_promotion         79080 non-null int64
description          79080 non-null object
category_name        79080 non-null object
manufacturer_name    79080 non-null object
order_value          79080 non-null float64
num_returns          79080 non-null float64
dtypes: float64(7), int64(3), object(5)
memory usage: 9.7+ MB


## Dummies for categorical parameters

In [85]:
df.order_date=df.order_date.str.replace('\d\/16\/\d+','Monday')
df.order_date=df.order_date.str.replace('\d\/17\/\d+','Tuesday')
df.order_date=df.order_date.str.replace('\d\/18\/\d+','Wednesday')
df.order_date=df.order_date.str.replace('\d\/19\/\d+','Thursday')
df.order_date=df.order_date.str.replace('\d\/20\/\d+','Friday')
df.order_date=df.order_date.str.replace('\d\/21\/\d+','Saturday')
df.order_date=df.order_date.str.replace('\d\/22\/\d+','Sunday')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79080 entries, 0 to 79079
Data columns (total 15 columns):
product_id           79080 non-null float64
order_date           79080 non-null object
order_time           79080 non-null object
customer_id          79080 non-null float64
zipcode              79080 non-null int64
quantity_ordered     79080 non-null int64
weight               79080 non-null float64
num_reviews          79080 non-null float64
onsite_price         79080 non-null float64
on_promotion         79080 non-null int64
description          79080 non-null object
category_name        79080 non-null object
manufacturer_name    79080 non-null object
order_value          79080 non-null float64
num_returns          79080 non-null float64
dtypes: float64(7), int64(3), object(5)
memory usage: 9.7+ MB


In [86]:
df2=pd.concat([df,pd.get_dummies(df.order_date)],axis=1)
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79080 entries, 0 to 79079
Data columns (total 22 columns):
product_id           79080 non-null float64
order_date           79080 non-null object
order_time           79080 non-null object
customer_id          79080 non-null float64
zipcode              79080 non-null int64
quantity_ordered     79080 non-null int64
weight               79080 non-null float64
num_reviews          79080 non-null float64
onsite_price         79080 non-null float64
on_promotion         79080 non-null int64
description          79080 non-null object
category_name        79080 non-null object
manufacturer_name    79080 non-null object
order_value          79080 non-null float64
num_returns          79080 non-null float64
Friday               79080 non-null uint8
Monday               79080 non-null uint8
Saturday             79080 non-null uint8
Sunday               79080 non-null uint8
Thursday             79080 non-null uint8
Tuesday              79080 non-n

In [87]:
for i in range(0,12):
    df2.order_time=df2.order_time.str.replace('^'+str(i)+':\S+','morning')
for i in range(12,18):
    df2.order_time=df2.order_time.str.replace('^'+str(i)+':\S+','afternoon')
for i in range(18,24):  
    df2.order_time=df2.order_time.str.replace('^'+str(i)+':\S+','evening')
df2.order_time.value_counts()    

afternoon    28379
evening      27860
morning      22841
Name: order_time, dtype: int64

In [88]:
df2=pd.concat([df2,pd.get_dummies(df2.order_time)],axis=1)
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79080 entries, 0 to 79079
Data columns (total 25 columns):
product_id           79080 non-null float64
order_date           79080 non-null object
order_time           79080 non-null object
customer_id          79080 non-null float64
zipcode              79080 non-null int64
quantity_ordered     79080 non-null int64
weight               79080 non-null float64
num_reviews          79080 non-null float64
onsite_price         79080 non-null float64
on_promotion         79080 non-null int64
description          79080 non-null object
category_name        79080 non-null object
manufacturer_name    79080 non-null object
order_value          79080 non-null float64
num_returns          79080 non-null float64
Friday               79080 non-null uint8
Monday               79080 non-null uint8
Saturday             79080 non-null uint8
Sunday               79080 non-null uint8
Thursday             79080 non-null uint8
Tuesday              79080 non-n

In [89]:
df2['description_length'] = df2.description.str.len()
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79080 entries, 0 to 79079
Data columns (total 26 columns):
product_id            79080 non-null float64
order_date            79080 non-null object
order_time            79080 non-null object
customer_id           79080 non-null float64
zipcode               79080 non-null int64
quantity_ordered      79080 non-null int64
weight                79080 non-null float64
num_reviews           79080 non-null float64
onsite_price          79080 non-null float64
on_promotion          79080 non-null int64
description           79080 non-null object
category_name         79080 non-null object
manufacturer_name     79080 non-null object
order_value           79080 non-null float64
num_returns           79080 non-null float64
Friday                79080 non-null uint8
Monday                79080 non-null uint8
Saturday              79080 non-null uint8
Sunday                79080 non-null uint8
Thursday              79080 non-null uint8
Tuesday     

In [90]:
category_count = df2.groupby('category_name')['description_length'].count().reset_index(name = 'category_count')
manu_count = df2.groupby('manufacturer_name')['description_length'].count().reset_index(name = 'manu_count')


In [91]:
df3 = pd.merge(df2,category_count,on='category_name')
df3 = pd.merge(df3,manu_count,on='manufacturer_name')
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79080 entries, 0 to 79079
Data columns (total 28 columns):
product_id            79080 non-null float64
order_date            79080 non-null object
order_time            79080 non-null object
customer_id           79080 non-null float64
zipcode               79080 non-null int64
quantity_ordered      79080 non-null int64
weight                79080 non-null float64
num_reviews           79080 non-null float64
onsite_price          79080 non-null float64
on_promotion          79080 non-null int64
description           79080 non-null object
category_name         79080 non-null object
manufacturer_name     79080 non-null object
order_value           79080 non-null float64
num_returns           79080 non-null float64
Friday                79080 non-null uint8
Monday                79080 non-null uint8
Saturday              79080 non-null uint8
Sunday                79080 non-null uint8
Thursday              79080 non-null uint8
Tuesday     

In [92]:
df3['category_rate'] = df3.category_count/(len(df3))
df3['manufacturer_rate'] = df3.manu_count/(len(df3))
df3.head()

,product_id,order_date,order_time,customer_id,zipcode,quantity_ordered,weight,num_reviews,onsite_price,on_promotion,...,Tuesday,Wednesday,afternoon,evening,morning,description_length,category_count,manu_count,category_rate,manufacturer_rate
0,7.842510e+15,Thursday,afternoon,6.231540e+18,61832,1,62.0,511.0,157.31,1,...,0,0,1,0,0,300,154,624,0.001947,0.007891
1,7.842510e+15,Wednesday,afternoon,3.839240e+18,8858,1,62.0,511.0,157.31,1,...,0,1,1,0,0,300,154,624,0.001947,0.007891
2,7.842510e+15,Saturday,morning,4.570650e+18,28054,1,62.0,511.0,157.31,1,...,0,0,0,0,1,300,154,624,0.001947,0.007891
3,7.842510e+15,Wednesday,evening,2.495660e+18,78233,1,62.0,511.0,157.31,1,...,0,1,0,1,0,300,154,624,0.001947,0.007891
4,7.842510e+15,Sunday,morning,7.211280e+18,7036,1,62.0,511.0,157.31,1,...,0,0,0,0,1,300,154,624,0.001947,0.007891


In [94]:
data = df3.drop(columns=['order_date','order_time','description','category_name','manufacturer_name','product_id','customer_id'])
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79080 entries, 0 to 79079
Data columns (total 23 columns):
zipcode               79080 non-null int64
quantity_ordered      79080 non-null int64
weight                79080 non-null float64
num_reviews           79080 non-null float64
onsite_price          79080 non-null float64
on_promotion          79080 non-null int64
order_value           79080 non-null float64
num_returns           79080 non-null float64
Friday                79080 non-null uint8
Monday                79080 non-null uint8
Saturday              79080 non-null uint8
Sunday                79080 non-null uint8
Thursday              79080 non-null uint8
Tuesday               79080 non-null uint8
Wednesday             79080 non-null uint8
afternoon             79080 non-null uint8
evening               79080 non-null uint8
morning               79080 non-null uint8
description_length    79080 non-null int64
category_count        79080 non-null int64
manu_count           

In [95]:
data.to_csv('regression_data.csv',index=False)

linear regression, random forest, neural 

# AutoML regression

## TPOT

In [2]:
from tpot import TPOTRegressor
from sklearn.model_selection import train_test_split
import pandas as pd

/anaconda3/lib/python3.6/site-packages/deap/tools/_hypervolume/pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap_external.py:426: ImportWarning: Not importing directory /anaconda3/lib/python3.6/site-packages/mpl_toolkits: missing __init__
  _warnings.warn(msg.format(portions[0]

In [3]:
df = pd.read_csv('regression_data.csv',)
df.head()

,product_id,customer_id,zipcode,quantity_ordered,weight,num_reviews,onsite_price,on_promotion,order_value,num_returns,...,Sunday,Thursday,Tuesday,Wednesday,afternoon,evening,morning,description_length,category_rate,manufacturer_rate
0,7.842510e+15,6.231540e+18,61832,1,62.0,511.0,157.31,1,157.31,2350.000000,...,0,1,0,0,1,0,0,300,0.001947,0.007891
1,7.842510e+15,3.839240e+18,8858,1,62.0,511.0,157.31,1,157.31,63.333333,...,0,0,0,1,1,0,0,300,0.001947,0.007891
2,7.842510e+15,4.570650e+18,28054,1,62.0,511.0,157.31,1,157.31,2795.000000,...,0,0,0,0,0,0,1,300,0.001947,0.007891
3,7.842510e+15,2.495660e+18,78233,1,62.0,511.0,157.31,1,157.31,3675.000000,...,0,0,0,1,0,1,0,300,0.001947,0.007891
4,7.842510e+15,7.211280e+18,7036,1,62.0,511.0,157.31,1,157.31,3731.666667,...,1,0,0,0,0,0,1,300,0.001947,0.007891


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79080 entries, 0 to 79079
Data columns (total 23 columns):
product_id            79080 non-null float64
customer_id           79080 non-null float64
zipcode               79080 non-null int64
quantity_ordered      79080 non-null int64
weight                79080 non-null float64
num_reviews           79080 non-null float64
onsite_price          79080 non-null float64
on_promotion          79080 non-null int64
order_value           79080 non-null float64
num_returns           79080 non-null float64
Friday                79080 non-null int64
Monday                79080 non-null int64
Saturday              79080 non-null int64
Sunday                79080 non-null int64
Thursday              79080 non-null int64
Tuesday               79080 non-null int64
Wednesday             79080 non-null int64
afternoon             79080 non-null int64
evening               79080 non-null int64
morning               79080 non-null int64
description_lengt

In [5]:
features = df.drop(columns=['product_id','customer_id','order_value']).values
target = df['order_value'].values

In [6]:
X_train, X_test, y_train, y_test = train_test_split(features, target,
                                                    train_size=0.75, test_size=0.25)
tpot = TPOTRegressor(generations=5, population_size=20, verbosity=2)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


Generation 1 - Current best internal CV score: -5765.543636130892
Generation 2 - Current best internal CV score: -4982.750818925716
Generation 3 - Current best internal CV score: -4982.750818925716
Generation 4 - Current best internal CV score: -2639.0953017832867
Generation 5 - Current best internal CV score: -2639.0953017832867

Best pipeline: RandomForestRegressor(MaxAbsScaler(input_matrix), bootstrap=False, max_features=0.9000000000000001, min_samples_leaf=1, min_samples_split=3, n_estimators=100)
-32309.805948525587


## H2O

In [8]:
import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,2 hours 2 mins
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.0.2
H2O cluster version age:,22 days
H2O cluster name:,H2O_from_python_jiaxinglong_pa8f6x
H2O cluster total nodes:,1
H2O cluster free memory:,3.999 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


In [15]:
df = h2o.import_file('regression_data.csv')
x = df.columns
y='order_value'
x.remove('product_id')
x.remove('customer_id')
x.remove('order_value')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [16]:
x

['zipcode',
 'quantity_ordered',
 'weight',
 'num_reviews',
 'onsite_price',
 'on_promotion',
 'num_returns',
 'Friday',
 'Monday',
 'Saturday',
 'Sunday',
 'Thursday',
 'Tuesday',
 'Wednesday',
 'afternoon',
 'evening',
 'morning',
 'description_length',
 'category_rate',
 'manufacturer_rate']

In [17]:
splits = df.split_frame(ratios=[0.8],seed=1)
train = splits[0]
test = splits[1]

In [20]:
aml = H2OAutoML(max_models = 20, seed = 1)
aml.train(x=x,y=y,training_frame = train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [23]:
aml.leader

Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  XGBoost_1_AutoML_20181214_145923


ModelMetricsRegression: xgboost
** Reported on train data. **

MSE: 254.3326720456985
RMSE: 15.947810885688936
MAE: 5.940524336998396
RMSLE: NaN
Mean Residual Deviance: 254.3326720456985

ModelMetricsRegression: xgboost
** Reported on validation data. **

MSE: 9537.443917306739
RMSE: 97.65983779070463
MAE: 9.057374964284229
RMSLE: 0.08488215539378789
Mean Residual Deviance: 9537.443917306739

ModelMetricsRegression: xgboost
** Reported on cross-validation data. **

MSE: 4961.4838295815725
RMSE: 70.43780114101783
MAE: 8.260402141480078
RMSLE: NaN
Mean Residual Deviance: 4961.4838295815725
Cross-Validation Metrics Summary: 


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
mae,8.260397,0.5291052,7.780088,9.038766,8.641425,7.030009,8.811697
mean_residual_deviance,4961.55,2242.885,3982.1016,2190.7256,2524.5454,10935.15,5175.226
mse,4961.55,2242.885,3982.1016,2190.7256,2524.5454,10935.15,5175.226
r2,0.9868528,0.0059619,0.9902129,0.992779,0.9929862,0.9704098,0.9878761
residual_deviance,4961.55,2242.885,3982.1016,2190.7256,2524.5454,10935.15,5175.226
rmse,67.33285,14.625955,63.103893,46.805187,50.244854,104.5712660,71.93904
rmsle,0.0949494,0.0142133,NaN,0.1108403,NaN,0.0790584,NaN


Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance,validation_rmse,validation_mae,validation_deviance
,2018-12-14 15:02:38,1 min 49.017 sec,0.0,660.0275813,249.5952455,435636.4081326,697.1502190,255.8036428,486018.4278448
,2018-12-14 15:02:38,1 min 49.311 sec,5.0,553.1943663,194.0291643,306024.0069105,589.3282965,199.6782389,347307.8410207
,2018-12-14 15:02:39,1 min 49.542 sec,10.0,448.6032585,151.6151959,201244.8835807,490.4780532,157.0974636,240568.7207149
,2018-12-14 15:02:39,1 min 49.774 sec,15.0,360.8058686,118.8351682,130180.8748259,403.2317225,123.7121781,162595.8220467
,2018-12-14 15:02:39,1 min 50.011 sec,20.0,299.0992609,93.2214325,89460.3678678,341.9871399,97.5276388,116955.2038767
---,---,---,---,---,---,---,---,---,---
,2018-12-14 15:02:58,2 min 8.567 sec,300.0,16.4490628,6.0968291,270.5716685,97.7678747,9.1907288,9558.5573224
,2018-12-14 15:02:58,2 min 9.036 sec,305.0,16.2622309,6.0352586,264.4601536,97.6506953,9.1285887,9535.6582841
,2018-12-14 15:02:59,2 min 9.553 sec,310.0,16.1023856,5.9912976,259.2868214,97.7296086,9.0984792,9551.0763977
,2018-12-14 15:02:59,2 min 10.083 sec,315.0,15.9563545,5.9439770,254.6052491,97.6545931,9.0560472,9536.4195515



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
onsite_price,2128.0,1.0,0.3279901
weight,1209.0,0.5681391,0.1863440
quantity_ordered,1154.0,0.5422932,0.1778668
category_rate,427.0,0.2006579,0.0658138
zipcode,377.0,0.1771617,0.0581073
manufacturer_rate,246.0,0.1156015,0.0379162
num_returns,219.0,0.1029135,0.0337546
num_reviews,215.0,0.1010338,0.0331381
description_length,209.0,0.0982143,0.0322133
Monday,159.0,0.0747180,0.0245068


In [24]:
import h2o
from h2o.automl import H2OAutoML
import pandas as pd


In [29]:
df = pd.DataFrame(data={
    'data':'Saturday',
    'time': 'afternoon',
    'BASKETLOADPAGE_total':['0'],
    'CATEGORYDEPARTMENT_total':['0'],
    'CATEGORYQUICKBROWSE_total':['0'],
    'CATEGORYSTANDARD_total':['0']
    'CHECKOUTBASKET_total':['0']
    'CHECKOUTPRODUCTUPSELLGROUP_total':['0']
    'DAILYSALESCLOSEOUTPAGE_total':['0']
    'DAILYSALESEVENTPAGE_total':['0']
    'DAILYSALESMAINPAGE_total':['0']
    'DAILYSALESPRODUCTPAGE_total':['0']
    'DAILYSALESSOLREVENTPAGE_total':['0']
    'HOMEPAGE_total':['0']
    'KEYWORDSEARCH_total':['0']
    'PRODUCTKIT_total':['0']
    'PRODUCTOPTIONSKU_total':['0']
    'PRODUCTSIMPLESKU_total':['0']
    'SAVETOBOARDMODALSCREEN_total':['0']
    'STLLANDINGPAGE_total':['0']
    'SUPERBROWSECATEGORY_total':['0']
    'SUPERBROWSECATEGORY1ATTRIBUTES_total':['0']
    'SUPERBROWSECATEGORY2ATTRIBUTES_total':['0']
    'SUPERBROWSECATEGORY3ATTRIBUTES_total':['0']
    'SUPERBROWSECATEGORY4+ATTRIBUTES_total':['0']
    'SUPERBROWSEHOTDEALS_total':['0']    
})

SyntaxError: invalid syntax (<ipython-input-29-6ba890a750bd>, line 8)

In [35]:
columns=['data','time',
         'BASKETLOADPAGE_total','CATEGORYDEPARTMENT_total','CATEGORYQUICKBROWSE_total','CATEGORYSTANDARD_total',
    'CHECKOUTBASKET_total','CHECKOUTPRODUCTUPSELLGROUP_total','DAILYSALESCLOSEOUTPAGE_total',
    'DAILYSALESEVENTPAGE_total',
    'DAILYSALESMAINPAGE_total',
    'DAILYSALESPRODUCTPAGE_total',
    'DAILYSALESSOLREVENTPAGE_total',
    'HOMEPAGE_total',
    'KEYWORDSEARCH_total',
    'PRODUCTKIT_total',
    'PRODUCTOPTIONSKU_total',
    'PRODUCTSIMPLESKU_total',
    'SAVETOBOARDMODALSCREEN_total',
    'STLLANDINGPAGE_total',
    'SUPERBROWSECATEGORY_total',
    'SUPERBROWSECATEGORY1ATTRIBUTES_total',
    'SUPERBROWSECATEGORY2ATTRIBUTES_total',
    'SUPERBROWSECATEGORY3ATTRIBUTES_total',
    'SUPERBROWSECATEGORY4+ATTRIBUTES_total',
    'SUPERBROWSEHOTDEALS_total']
df = pd.DataFrame(columns=columns)
df.loc[0]=['Saturday','afternoon','0','0','0','0','0','0','0','0','0','0','0','0','1','0','0','0','0','0','0','0','0','0','0','0']

In [48]:
page = []
for i in range(24):
    page.append('0')
page[12] = '1'    
listInput = []
date = 'Saturday'
time = 'afternoon'
listInput.append(date)
listInput.append(time)
for i in range(24):
    listInput.append(page[i])
df.loc[0]=listInput

In [50]:
df.T

,0
data,Saturday
time,afternoon
BASKETLOADPAGE_total,0
CATEGORYDEPARTMENT_total,0
CATEGORYQUICKBROWSE_total,0
CATEGORYSTANDARD_total,0
CHECKOUTBASKET_total,0
CHECKOUTPRODUCTUPSELLGROUP_total,0
DAILYSALESCLOSEOUTPAGE_total,0
DAILYSALESEVENTPAGE_total,0


In [51]:
class_model = h2o.load_model('StackedEnsemble_AllModels_AutoML_20181214_163440')

result = class_model.predict(h2o.H2OFrame(df))

Parse progress: |█████████████████████████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%


/anaconda3/lib/python3.6/site-packages/h2o/job.py:69: UserWarning: Test/Validation dataset is missing column 'date': substituting in a column of NaN
  warnings.warn(w)
/anaconda3/lib/python3.6/site-packages/h2o/job.py:69: UserWarning: Test/Validation dataset is missing column 'C1': substituting in a column of NaN
  warnings.warn(w)


In [55]:
string = str(result)

predict,p0,p1
1,0.446365,0.553635


In [62]:
result.as_data_frame().to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>predict</th>\n      <th>p0</th>\n      <th>p1</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>1</td>\n      <td>0.446365</td>\n      <td>0.553635</td>\n    </tr>\n  </tbody>\n</table>'

In [70]:
from sklearn.ensemble import RandomForestRegressor

clf = RandomForestRegressor(
    bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=1000, n_jobs=1, oob_score=False, random_state=42,
           verbose=0, warm_start=False).fit()


TypeError: fit() missing 2 required positional arguments: 'X' and 'y'

In [101]:
columns=['zipcode',              
'quantity_ordered',    
'weight',                
'num_reviews',            
'onsite_price',           
'on_promotion',                    
'num_returns',           
'Friday',              
'Monday',               
'Saturday',            
'Sunday',               
'Thursday',             
'Tuesday',              
'Wednesday',            
'afternoon',            
'evening',              
'morning',              
'description_length',   
'category_count',       
'manu_count',           
'category_rating',
'manufacturer_rating']
df = pd.DataFrame(columns=columns)
inputlist = []
for i in range(22):
    inputlist.append('0')
df.loc[0] = inputlist
df

,zipcode,quantity_ordered,weight,num_reviews,onsite_price,on_promotion,num_returns,Friday,Monday,Saturday,...,Tuesday,Wednesday,afternoon,evening,morning,description_length,category_count,manu_count,category_rating,manufacturer_rating
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [68]:
result = clf.predict(df)
result

NotFittedError: This RandomForestRegressor instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

In [98]:
data = pd.read_csv('regression_data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79080 entries, 0 to 79079
Data columns (total 23 columns):
zipcode               79080 non-null int64
quantity_ordered      79080 non-null int64
weight                79080 non-null float64
num_reviews           79080 non-null float64
onsite_price          79080 non-null float64
on_promotion          79080 non-null int64
order_value           79080 non-null float64
num_returns           79080 non-null float64
Friday                79080 non-null int64
Monday                79080 non-null int64
Saturday              79080 non-null int64
Sunday                79080 non-null int64
Thursday              79080 non-null int64
Tuesday               79080 non-null int64
Wednesday             79080 non-null int64
afternoon             79080 non-null int64
evening               79080 non-null int64
morning               79080 non-null int64
description_length    79080 non-null int64
category_count        79080 non-null int64
manu_count           

In [102]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1 entries, 0 to 0
Data columns (total 22 columns):
zipcode                1 non-null object
quantity_ordered       1 non-null object
weight                 1 non-null object
num_reviews            1 non-null object
onsite_price           1 non-null object
on_promotion           1 non-null object
num_returns            1 non-null object
Friday                 1 non-null object
Monday                 1 non-null object
Saturday               1 non-null object
Sunday                 1 non-null object
Thursday               1 non-null object
Tuesday                1 non-null object
Wednesday              1 non-null object
afternoon              1 non-null object
evening                1 non-null object
morning                1 non-null object
description_length     1 non-null object
category_count         1 non-null object
manu_count             1 non-null object
category_rating        1 non-null object
manufacturer_rating    1 non-null obje

In [99]:
features = data.drop(columns=['order_value']).values
target = data.order_value.values
X_train, X_test, y_train, y_test = train_test_split(features, target, train_size=0.75, test_size=0.25)
clf = RandomForestRegressor(n_estimators= 1000, random_state= 42)
clf.fit(X_train, y_train)
reg_result = clf.predict(df)
reg_result

ValueError: could not convert string to float: 'Cuisinox'

In [103]:
reg_result = clf.predict(df)

In [104]:
reg_result

array([0.5361])

In [108]:
len(X_train[0])

22

In [112]:
clf.predict(df)

array([171.99])

In [110]:
df.loc[0] = X_train[0]

In [111]:
df

,zipcode,quantity_ordered,weight,num_reviews,onsite_price,on_promotion,num_returns,Friday,Monday,Saturday,...,Tuesday,Wednesday,afternoon,evening,morning,description_length,category_count,manu_count,category_rating,manufacturer_rating
0,47630,1,2.9,4,171.99,0,2858.33,0,0,0,...,0,0,0,0,1,273,227,373,0.00287051,0.00471674


In [113]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1 entries, 0 to 0
Data columns (total 22 columns):
zipcode                1 non-null object
quantity_ordered       1 non-null object
weight                 1 non-null object
num_reviews            1 non-null object
onsite_price           1 non-null object
on_promotion           1 non-null object
num_returns            1 non-null object
Friday                 1 non-null object
Monday                 1 non-null object
Saturday               1 non-null object
Sunday                 1 non-null object
Thursday               1 non-null object
Tuesday                1 non-null object
Wednesday              1 non-null object
afternoon              1 non-null object
evening                1 non-null object
morning                1 non-null object
description_length     1 non-null object
category_count         1 non-null object
manu_count             1 non-null object
category_rating        1 non-null object
manufacturer_rating    1 non-null obje

In [122]:
result = clf.predict(df)

In [126]:
result[0]

171.98999999999953

In [115]:
list1 = []
list1.append(['0','3','1'])

In [116]:
list1

[['0', '3', '1']]

In [117]:
list2=[]
list2.append('0')
list2.append('3')
list2

['0', '3']

In [118]:
list3 = []
list3.append('Friday' == 'Friday' ? 0:1)
list3

SyntaxError: invalid syntax (<ipython-input-118-8b3fdc569ca6>, line 2)

In [121]:
list4 = []
day = ['Friday',              
        'Monday',               
        'Saturday',            
        'Sunday',               
        'Thursday',             
        'Tuesday',              
        'Wednesday']
for i in day:
    if 'Wednesday' == i:
        list4.append('1')
    else:
        list4.append('0')

list4

['0', '0', '0', '0', '0', '0', '1']